# Source Analysis in TF Domain



In [ ]:
import numpy as np
import mne
mne.set_config('MNE_BROWSER_BACKEND', 'matplotlib')
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

from pmf import *
from utils import * 

# EEG Import

In [ ]:
raw = mne.io.read_raw_eeglab(input_fname = "data.set")
raw.info

In [ ]:
raw.plot(n_channels=len(raw));

# X = WZ

X is regarded as mixtures, which is an M by N matrix.

W represents a linear mapping, which is full column rank M by r matrix. 

Z represents source signals. It is r by N matrix.

In [ ]:
raw_avg = raw.copy()
raw_avg.load_data()
raw_avg.set_eeg_reference()

events, event_id = mne.events_from_annotations(raw_avg)
tmin = -0.01  # Start time before event
tmax = 2.5  # End time after event
epochs = mne.Epochs(raw_avg, events, event_id=2, tmin=tmin, tmax=tmax, preload=True)
X = epochs.average().data;

In [ ]:
r = 32
M,N = X.shape

In [ ]:
# PMF EXECUTION FOR ALL DATA

W, Z = PMF(raw, X, r = r, NumberofIterations = 2801, muv=10**(9.), epsv=1e-9, tuning=False, tmin = tmin, tmax = tmax)

In [ ]:
#Calculate the RVs

dip_pmf_rvs, dip_pmf_gofs = calculate_rvs(raw,W,M,N,r,tmin = tmin, tmax = tmax)

In [ ]:
#Plot
pmf_plot(r, W, raw, dip_pmf_rvs, treshold=0.15)

In [ ]:
single_events = np.zeros([1,3],np.int64)
single_events[0][2] = 1

ch_names = []
for i in range(r):
    ch_names.append(f"Source {i}")

ch_types = ["eeg"] * r
Z_info = mne.create_info(ch_names, ch_types=ch_types, sfreq=1024)
Z_mne = mne.io.RawArray(Z, Z_info)

square_epochs = mne.Epochs(Z_mne, single_events, event_id=1, tmin=0, tmax=tmax-tmin, preload=True, baseline=None)

square_epochs

In [ ]:
freqs = np.logspace(*np.log10([6, 35]), num=8)
n_cycles = freqs / 2.0  # different number of cycle per frequency
power, itc = mne.time_frequency.tfr_morlet(square_epochs,
    freqs=freqs,
    n_cycles=n_cycles,
    average=True,
    return_itc=True,
    decim=3,
)

In [ ]:
power.plot(title="auto")